In [11]:
%%capture
!pip install wikidataintegrator

In [2]:
from wikidataintegrator import wdi_core, wdi_login 
##from getpass import getpass
import pandas as pd
from copy import deepcopy

In [3]:
wikibase = "http://localhost:8080"
api = "http://localhost:8080/w/api.php"
sparql = "http://localhost:8282/proxy/wdqs/bigdata/namespace/wdq/sparql"
entityUri = "http://mediawiki.svc/entity/"  # this is used to strip it from results! 

WBUSER = "SafAdmin"
#WBPASS = getpass()
WBPASS = "ExampleAdminPassword"
login = wdi_login.WDLogin(WBUSER, WBPASS, mediawiki_api_url=api)

In [3]:
data_load_10_dead = pd.read_excel("Export_SAF_LUX_10_dead can dance.xlsx", header=0)
data_load_anlux = pd.read_excel("Untitled.xlsx", header=0)
pd.set_option('display.max_columns', None)


In [4]:
data_load_10_dead.columns.difference(data_load_anlux.columns)

Index([], dtype='object')

In [5]:
data_load_anlux.columns.difference(data_load_10_dead.columns)

Index([], dtype='object')

In [6]:
localEntityEngine = wdi_core.WDItemEngine.wikibase_item_engine_factory(api, sparql)

#person_item = localEntityEngine(new_item = True)
#person_item.set_label("E21 Person", lang="en")
#person_item.write(login)

In [7]:
qid = dict()
query = "SELECT ?item ?label WHERE {{ ?item rdfs:label ?label }}"
wdi_core.WDItemEngine.execute_sparql_query(query, as_dataframe = True, endpoint=sparql)
for index, row in wdi_core.WDItemEngine.execute_sparql_query(query, as_dataframe = True, endpoint=sparql).iterrows():
    qid[row["label"]] = row["item"].replace(entityUri, "")

print('qid now contains {} items'.format(len(qid)))

qid now contains 481 items


# Importing "Export_SAF_LUX_10_dead can dance.xlsx"

In [10]:
dataload = data_load_10_dead
for index, row in dataload.iterrows():
    statements = []
    references = []
    good_reference = [
        {qid["source of information - url"]: None, qid["source of information - text"]: None},
        {qid["source of information - text"]: None, qid["source of information - url"]: None},
        {qid["source of information - url"]: None}, 
        {qid["source of information - text"]: None}
    ]

    if not pd.isnull(row["Sources of information:Url"]):
        # TODO: can contain either a URL that starts with http:// or more than one, once of which is missing the protocol
        # e.g. "www.a-z.lu/BIBNET:ALEPH_LUX01001456372;www.a-z.lu/BIBNET:ALEPH_LUX30000001060"
        # or www.a-z.lu/BIBNET:ALEPH_LUX01001137708;https://www.greatalexweyer.lu/
        if row["Sources of information:Url"].startswith("http"):
            
            # Set statements            
            # is a person (DR: the query above didn't distinguish between entities (Qx) and properties (Px))
            # so everything is in the same dict
            statements.append(wdi_core.WDItemID(value = qid["E21 Person"], prop_nr = qid['instance of']))

            # Set references
            references.append(
                wdi_core.WDUrl(
                    value=row["Sources of information:Url"], 
                    prop_nr=qid["source of information - url"], 
                    is_reference=True
                )
            )
            references.append(wdi_core.WDString(value=row["Sources of information:Text"], prop_nr=qid["source of information - text"], is_reference=True))

            # name qualifiers
            qualifiers_name = []
                     
            ## type of naming (direct/indirect, see also CL3)
            if str(row["Heading:Type"]) == "1": 
                qualifiers_name.append(wdi_core.WDItemID(value = qid["indirect name form"], prop_nr = qid["name format"], is_qualifier=True))
            elif str(row["Heading:Type"]) == "0":
                qualifiers_name.append(wdi_core.WDItemID(value=qid["direct name form"], prop_nr=qid["name format"], is_qualifier=True))
            else:
                print('Unknown name format key "{}"'.format(row['Heading:Type']))
            
            ## numeration 
            if not pd.isnull(row["Heading:Numeration"]):
                qualifiers_name.append(wdi_core.WDString(value = row["Heading:Numeration"], prop_nr = qid["numeration"], is_qualifier=True))
            
            ## title
            if not pd.isnull(row["Heading:Title"]):
                qualifiers_name.append(wdi_core.WDString(value = row["Heading:Title"], prop_nr=qid["title"], is_qualifier=True))
            
            # add name to the list of statements, using the qualifiers and references created above
            statements.append(
                wdi_core.WDString(
                    value = row["Heading:Name"], 
                    prop_nr = qid["name"], 
                    qualifiers = deepcopy(qualifiers_name), 
                    references = [deepcopy(references)]
                )
            )     

            if not pd.isnull(row["Alternative / other form(s) of name:Name"]):
                alternative_name_statement = []
                alternative_names = row["Alternative / other form(s) of name:Name"].split(";")
                print("Adding alternative name: {}".format(row["Alternative / other form(s) of name:Type"]))
                an_types = str(row["Alternative / other form(s) of name:Type"]).split(";")
                if not pd.isnull(row["Alternative / other form(s) of name:Numeration"]):
                    an_numerations = row["Alternative / other form(s) of name:Numeration"].split(";")
                else: 
                    an_numerations = None
                
                if not pd.isnull(row["Alternative / other form(s) of name:Title"]):
                    an_titles = row["Alternative / other form(s) of name:Title"].split(";")
                    print('Titles for alternative names are "{}"'.format(an_titles))
                else:
                    an_titles = dict()
                an_index = 0
                for an in alternative_names:
                    qualifiers_aname = []

                    ## type
                    if len(an_types) > 0:
                        if an_index < len(an_types):
                            if an_types[an_index] == "1":
                                qualifiers_aname.append(wdi_core.WDItemID(value=qid["indirect name form"], prop_nr=qid["name format"], is_qualifier=True)) 
                            elif an_types[an_index] == "0":
                                qualifiers_aname.append(wdi_core.WDItemID(value=qid["direct name form"], prop_nr=qid["name format"], is_qualifier=True)) 
                    ## numeration 
                    if not pd.isnull(an_numerations):
                        if an_index in an_numerations.keys():
                            qualifiers_aname.append(wdi_core.WDString(value = an_numerations[an_index], prop_nr = qid["numeration"], is_qualifier=True))

                    if len(an_titles) > 0:
                        if an_index < len(an_titles):
                            qualifiers_aname.append(wdi_core.WDString(value = an_titles[an_index], prop_nr = qid["title"], is_qualifier=True))
                    
                    # add alternative name to statements
                    statements.append(
                        wdi_core.WDString(value=an, prop_nr = qid["alternative name"], qualifiers = deepcopy(qualifiers_aname), references=[deepcopy(references)])
                    )
                    an_index += 1
            
            # gender
            if not pd.isnull(row["Gender:Gender"]): 
                if str(int(row["Gender:Gender"])) == "1":
                    gender_statement = wdi_core.WDItemID(value = qid["male"], prop_nr=qid["gender"], references=[deepcopy(references)])
                elif str(int(row["Gender:Gender"])) == "2":
                    gender_statement = wdi_core.WDItemID(value = qid["female"], prop_nr=qid["gender"], references=[deepcopy(references)])
                elif str(int(row["Gender:Gender"])) == "0":
                    gender_statement = wdi_core.WDItemID(value = qid["not known"], prop_nr=qid["gender"], references=[deepcopy(references)])
                elif str(int(row["Gender:Gender"])) == "9":
                    gender_statement = wdi_core.WDItemID(value = qid["not applicable"], prop_nr=qid["gender"], references=[deepcopy(references)])

                statements.append(gender_statement)

            #birthdate
            #statements.append(wdi_core.WDEdtf(value=row["Associated identification information:Birth date"], prop_nr=qid["date of birth"]))
            #birthplace
            if not pd.isnull(row["Associated identification information:Birth place"]):
                statements.append(
                    wdi_core.WDString(
                        value = row["Associated identification information:Birth place"], 
                        prop_nr = qid["place of birth"], 
                        references = [deepcopy(references)]
                    )
                )

            # place of death
            if not pd.isnull(row["Associated identification information:Death place"]):
                statements.append(
                    wdi_core.WDString(
                        value = row["Associated identification information:Death place"], 
                        prop_nr = qid["place of death"], 
                        references = [deepcopy(references)]
                    )
                )
            
            # profession
            if not pd.isnull(row["Profession:Type"]):
                statements.append(wdi_core.WDString(value=row["Profession:Type"], prop_nr=qid["profession"], references=[deepcopy(references)]))

            if not pd.isnull(row["Activity:Type"]):
                activities = row["Activity:Type"].split(";")
            
            if not pd.isnull(row["Activity:Beginning"]):
                activities_start = str(row["Activity:Beginning"]).split(";")
                
            if not pd.isnull(row["Activity:End"]):
                activities_start = str(row["Activity:End"]).split(";")

            j = 0
            for activity in activities:
                statements.append(wdi_core.WDString(value=activity, prop_nr=qid["activity"], references=[deepcopy(references)]))
                # TODO add qualifiers once EDTF is supported   
                
            # internal identifier
            statements.append(
                wdi_core.WDExternalID(
                    value = str(row["Administration fields:Internal standard identifier"]), 
                    prop_nr=qid["AFL identifier"], 
                    references=[deepcopy(references)]
                )
            )

            # external identifiers
            identifiers = str(row["Administration fields:External standard identifier:ID"]).split(";")
            sources = str(row["Administration fields:External standard identifier:Name"]).split(";")

            if len(identifiers) != len(sources):
                print("ERROR: number of identifiers do not match number of sources")
                continue
            
            i = 0
            for identifier in identifiers:
                if sources[i] == "ARK":
                    statements.append(wdi_core.WDUrl(value=identifier, prop_nr=qid["ARK"], references=[deepcopy(references)]))
                elif sources[i] == "VIAF":
                    statements.append(wdi_core.WDExternalID(value=identifier, prop_nr=qid["VIAF"], references=[deepcopy(references)]))
                elif sources[i] == "ISNI":
                    statements.append(wdi_core.WDExternalID(value=identifier, prop_nr=qid["ISNI"], references=[deepcopy(references)]))
                print(identifier, sources[i])
                i +=1
            
            updated = False
            print('Now creating or updating "{}"'.format(row["Heading:Name"]))
            if row["Heading:Name"] not in qid.keys():
                item = localEntityEngine(new_item=True, data = statements)
            else:
                item = localEntityEngine(wd_item_id = qid[row["Heading:Name"]], data=statements, good_refs=good_reference)
                updated = True

            item.set_label(row["Heading:Name"], lang="lb")
            item.set_label(row["Heading:Name"], lang="de")
            item.set_label(row["Heading:Name"], lang="en")
            item.set_label(row["Heading:Name"], lang="fr")
            
            qid[row["Heading:Name"]] = item.write(login)
            print(
                'Person "{}" {} as entity "{}"'.format(
                    row["Heading:Name"], 
                    'updated' if updated else 'created', 
                    qid[row["Heading:Name"]]
                )
            )
        else:
            print('The source of information URL for "{}" does not start with http://'.format(row["Heading:Name"]))


The source of information URL for "Zorn, Wilhelm" does not start with http://
Adding alternative name: 0;1;0;0;1
0000 0000 4390 2515 ISNI
https://persist.lu/ark:70795/mb1387x74 ARK
77038289 VIAF
Now creating or updating "Zenner, Theodor"
Person "Zenner, Theodor" created as entity "Q835"
Adding alternative name: 0;1
https://persist.lu/ark:70795/nc0kv6rqb ARK
15145003276461300298 VIAF
Now creating or updating "Zenner, Jean"
Person "Zenner, Jean" created as entity "Q836"
The source of information URL for "Zanussi, Marc" does not start with http://
Adding alternative name: 0
0000 0003 8517 1599 ISNI
https://persist.lu/ark:70795/0153psjrq ARK
35462746 VIAF
Now creating or updating "Zanen, Jean-Pierre"
Person "Zanen, Jean-Pierre" created as entity "Q837"
Adding alternative name: 0;1
https://persist.lu/ark:70795/p87fq64br ARK
9779161098987429640006 VIAF
Now creating or updating "Wust, Jean-Pierre"
Person "Wust, Jean-Pierre" created as entity "Q838"
Adding alternative name: 1;1;1
https://persi

WDApiError: {'error': {'code': 'modification-failed', 'info': 'Must be no more than 400 characters long', 'messages': [{'name': 'wikibase-validator-too-long', 'parameters': ['400', 'Dictionnaire des auteurs luxe...'], 'html': {'*': 'Must be no more than 400 characters long'}}], '*': 'See http://localhost:8080/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}}

# ANLux data

In [55]:
import traceback
data_load = data_load_anlux
good_reference = [{qid["source of information - url"]: None, qid["source of information - text"]: None},
                      {qid["source of information - text"]: None, qid["source of information - url"]: None},
                      {qid["source of information - url"]: None}, 
                      {qid["source of information - text"]: None}]

for index, row in data_load.iterrows():
    statements = []
    references = []
    if not pd.isnull(row["Sources of information:Text"]):
        for reference in row["Sources of information:Text"].split('\n'):
            m = re.search(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)', reference)
            if m: 
                print(m.group(0))
                references.append(wdi_core.WDUrl(value=m.group(0), prop_nr=qid["source of information - url"], is_reference=True))
            else:
                references.append(wdi_core.WDString(value=reference, prop_nr=qid["source of information - text"], is_reference=True))
        # is a person 
        statements.append(wdi_core.WDItemID(value=qid["E21 Person"], prop_nr=qid['instance of']))
        # name qualifiers
        qualifiers_name =[]
        ## type
        if row["Heading:Type"] == "1": 
            qualifiers_name.append(wdi_core.WDItemID(value=qid["indirect name form"], prop_nr=qid["name format"], is_qualifier=True))
        elif row["Heading:Type"] == "0":
            qualifiers_name.append(wdi_core.WDItemID(value=qid["direct name form"], prop_nr=qid["name format"], is_qualifier=True))
        ## numeration 
        if not pd.isnull(row["Heading:Numeration"]):
            qualifiers_name.append(wdi_core.WDString(value=row["Heading:Numeration"], prop_nr=qid["numeration"], is_qualifier=True))
        ## title
        if not pd.isnull(row["Heading:Title"]):
            qualifiers_name.append(wdi_core.WDString(value=row["Heading:Title"], prop_nr=qid["title"], is_qualifier=True))
        # add name 
        statements.append(wdi_core.WDString(value=row["Heading:Name"], prop_nr=qid["name"], qualifiers=deepcopy(qualifiers_name), references=[deepcopy(references)]))     

        if not pd.isnull(row["Alternative / other form(s) of name:Name"]):
            alternative_name_statement = []
            alternative_names = row["Alternative / other form(s) of name:Name"].split(";")
            print(row["Alternative / other form(s) of name:Type"])
            an_types = str(row["Alternative / other form(s) of name:Type"]).split(";")
            if not pd.isnull(row["Alternative / other form(s) of name:Numeration"]):
                an_numerations = row["Alternative / other form(s) of name:Numeration"].split(";")
            else: 
                an_numerations = None
            if not pd.isnull(row["Alternative / other form(s) of name:Title"]):
                an_titles = row["Alternative / other form(s) of name:Title"].split(";")
                print(an_titles)
            else:
                an_titles = dict()
            index = 0
            for an in alternative_names:
                qualifiers_aname = []

                ## type
                if len(an_types)>0:
                    if index < len(an_types):
                        if an_types[index].strip() == "1":
                            qualifiers_aname.append(wdi_core.WDItemID(value=qid["indirect name form"], prop_nr=qid["name format"], is_qualifier=True)) 
                        elif an_types[index].strip() == "0":
                            qualifiers_aname.append(wdi_core.WDItemID(value=qid["direct name form"], prop_nr=qid["name format"], is_qualifier=True)) 
                ## numeration 
                if not pd.isnull(an_numerations):
                    if index in an_numerastions.keys():
                        qualifiers_aname.append(wdi_core.WDString(value=an_numeration[index], prop_nr=qid["numeration"], is_qualifier=True))

                if len(an_titles)>0:
                    if index < len(an_titles):
                        qualifiers_aname.append(wdi_core.WDString(value=an_titles[index], prop_nr=qid["title"], is_qualifier=True))
                # add name 
                statements.append(wdi_core.WDString(value=an.strip(), prop_nr=qid["alternative name"], qualifiers=deepcopy(qualifiers_aname), references=[deepcopy(references)]))
                index +=1

        #gender
        if not pd.isnull(row["Gender:Gender"]): 
            if str(int(row["Gender:Gender"])) == "1":
                statements.append(wdi_core.WDItemID(value=qid["male"], prop_nr=qid["gender"], references=[deepcopy(references)]))
            elif str(int(row["Gender:Gender"])) == "2":
                statements.append(wdi_core.WDItemID(value=qid["female"], prop_nr=qid["gender"], references=[deepcopy(references)]))
            elif str(int(row["Gender:Gender"])) == "0":
                statements.append(wdi_core.WDItemID(value=qid["not known"], prop_nr=qid["gender"], references=[deepcopy(references)]))
            elif str(int(row["Gender:Gender"])) == "9":
                statements.append(wdi_core.WDItemID(value=qid["not applicable"], prop_nr=qid["gender"], references=[deepcopy(references)]))

        #birthdate
        #statements.append(wdi_core.WDEdtf(value=row["Associated identification information:Birth date"], prop_nr=qid["date of birth"]))
        #birthplace
        if not pd.isnull(row["Associated identification information:Birth place"]):
            statements.append(wdi_core.WDString(value=row["Associated identification information:Birth place"], prop_nr=qid["place of birth"], references=[deepcopy(references)]))

        # place of death
        if not pd.isnull(row["Associated identification information:Death place"]):
            statements.append(wdi_core.WDString(value=row["Associated identification information:Death place"], prop_nr=qid["place of death"], references=[deepcopy(references)]))

        # profession
        if not pd.isnull(row["Profession:Type"]):
            statements.append(wdi_core.WDString(value=row["Profession:Type"], prop_nr=qid["profession"], references=[deepcopy(references)]))
        activities = []
        if not pd.isnull(row["Activity:Type"]):
            activities = row["Activity:Type"].split(";")

        if not pd.isnull(row["Activity:Beginning"]):
            activities_start = str(row["Activity:Beginning"]).split(";")

        if not pd.isnull(row["Activity:End"]):
            activities_start = str(row["Activity:End"]).split(";")

        j = 0
        for activity in activities:
            statements.append(wdi_core.WDString(value=activity, prop_nr=qid["activity"], references=[deepcopy(references)]))
            # TODO add qualifiers once EDTF is supported   

        # internal identifier
        statements.append(wdi_core.WDExternalID(value=str(row["Administration fields:Internal standard identifier"]), prop_nr=qid["AFL identifier"], references=[deepcopy(references)]))

        # external identifiers
        identifiers = str(row["Administration fields:External standard identifier:ID"]).split(";")
        sources = str(row["Administration fields:External standard identifier:Name"]).split(";")
        if len(identifiers) != len(sources):
            print("ERROR: number of identifiers do not match number of sources")
            continue
        i = 0
        for identifier in identifiers:
            if sources[i] == "ARK":
                statements.append(wdi_core.WDUrl(value=identifier, prop_nr=qid["ARK"], references=[deepcopy(references)]))
            elif sources[i] == "VIAF":
                statements.append(wdi_core.WDExternalID(value=identifier, prop_nr=qid["VIAF"], references=[deepcopy(references)]))
            elif sources[i] == "ISNI":
                statements.append(wdi_core.WDExternalID(value=identifier, prop_nr=qid["ISNI"], references=[deepcopy(references)]))
            print(identifier, sources[i])
            i +=1
        print(row["Heading:Name"])
        if row["Heading:Name"] not in qid.keys():
            item = localEntityEngine(new_item=True, data=statements)
        else:
            item = localEntityEngine(wd_item_id=qid[row["Heading:Name"]], data=statements, good_refs=good_reference, keep_good_ref_statements=True)

        item.set_label(row["Heading:Name"], lang="lb")
        item.set_label(row["Heading:Name"], lang="de")
        item.set_label(row["Heading:Name"], lang="en")
        item.set_label(row["Heading:Name"], lang="fr")
        
        try:
            qid[row["Heading:Name"]] = item.write(login)
        except:
            print("Failed: "+row["Heading:Name"])
            print("traceback: "+traceback.print_exc())
        print(row["Heading:Name"], qid[row["Heading:Name"]])

http://www.isni.org/isni/0000000082044322
https://de.wikipedia.org/wiki/Petrus_van_der_Aa
1
nan nan
Aa, Petrus van der
Aa, Petrus van der Q879
http://www.orval.be/fr/71/Peintre-Frere-Abraham-Gilson
1 ; 1
nan nan
Abraham d'Orval


KeyboardInterrupt: 

In [34]:
import re
for reference in row["Sources of information:Text"].split("\n"):
    m = re.search(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)', reference)
    if m: 
        print(m.group(0))

http://www.isni.org/isni/0000000082044322
https://de.wikipedia.org/wiki/Petrus_van_der_Aa


In [56]:
good_reference

[{'P16': None, 'P12': None},
 {'P12': None, 'P16': None},
 {'P16': None},
 {'P12': None}]

In [56]:
tjoepie = data_load[data_load["Heading:Name"] =="Willibrordus"]

In [112]:
for index, row in data_load.iterrows():
    print()

1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1


ValueError: cannot convert float NaN to integer

In [134]:
item.get_wd_json_representation()

{'claims': {'P1': [{'mainsnak': {'snaktype': 'value',
     'property': 'P1',
     'datavalue': {'value': {'entity-type': 'item',
       'numeric-id': 17,
       'id': 'Q17'},
      'type': 'wikibase-entityid'},
     'datatype': 'wikibase-item'},
    'type': 'statement',
    'rank': 'normal',
    'qualifiers': {},
    'qualifiers-order': [],
    'references': []}],
  'P17': [{'mainsnak': {'snaktype': 'value',
     'property': 'P17',
     'datavalue': {'value': 'R.R.', 'type': 'string'},
     'datatype': 'string'},
    'type': 'statement',
    'rank': 'normal',
    'qualifiers': {'P9': [{'snaktype': 'value',
       'property': 'P9',
       'datavalue': {'value': {'entity-type': 'item',
         'numeric-id': 15,
         'id': 'Q15'},
        'type': 'wikibase-entityid'},
       'datatype': 'wikibase-item'}]},
    'qualifiers-order': ['P9'],
    'references': [{'snaks': {'P16': [{'snaktype': 'value',
         'property': 'P16',
         'datavalue': {'value': 'http://www.autorenlexikon.l